# Exploratory Data Analysis (EDA) with SQL

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.

### Download datasets


In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>

In [1]:
!pip install sqlalchemy==1.3.9

### Connect to the database

Let us load the SQL extension and establish a connection with the database

In [2]:
!pip install ipython-sql

In [3]:
%load_ext sql

In [4]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
!pip install -q pandas

In [6]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [7]:
import pandas as pd
df = pd.read_csv("SpaceX.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

**Note:This below code is added to remove blank rows from table**

In [8]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [9]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

### Tasks
Write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

In [14]:
%%sql

SELECT *
    FROM sqlite_master
    WHERE type='table';

 * sqlite:///my_data1.db
Done.


type,name,tbl_name,rootpage,sql
table,SPACEXTBL,SPACEXTBL,2,"CREATE TABLE ""SPACEXTBL"" (""Date"" TEXT, ""Time (UTC)"" TEXT, ""Booster_Version"" TEXT, ""Launch_Site"" TEXT, ""Payload"" TEXT, ""PAYLOAD_MASS__KG_"" INTEGER, ""Orbit"" TEXT, ""Customer"" TEXT, ""Mission_Outcome"" TEXT, ""Landing_Outcome"" TEXT)"
table,SPACEXTABLE,SPACEXTABLE,6,"CREATE TABLE SPACEXTABLE( Date TEXT, ""Time (UTC)"" TEXT, Booster_Version TEXT, Launch_Site TEXT, Payload TEXT, PAYLOAD_MASS__KG_ INT, Orbit TEXT, Customer TEXT, Mission_Outcome TEXT, Landing_Outcome TEXT)"


##### Display the names of the unique launch sites  in the space mission

In [26]:
%%sql
SELECT DISTINCT(Launch_Site) FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#####  Display 5 records where launch sites begin with the string 'CCA' 

In [27]:
%%sql
SELECT * FROM SPACEXTBL
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


##### Display the total payload mass carried by boosters launched by NASA (CRS)

In [33]:
%%sql
SELECT Customer, SUM(PAYLOAD_MASS__KG_) FROM SPACEXTBL
WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


Customer,SUM(PAYLOAD_MASS__KG_)
NASA (CRS),45596



##### Display average payload mass carried by booster version F9 v1.1

In [34]:
%%sql
SELECT Booster_Version, AVG(PAYLOAD_MASS__KG_) FROM SPACEXTBL
WHERE Booster_Version LIKE 'F9 v1.1%';

 * sqlite:///my_data1.db
Done.


Booster_Version,AVG(PAYLOAD_MASS__KG_)
F9 v1.1 B1003,2534.6666666666665


##### List the date when the first succesful landing outcome in ground pad was acheived.

In [39]:
%%sql
SELECT MIN(Date) FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


MIN(Date)
2015-12-22


##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [42]:
%%sql
SELECT Booster_Version, PAYLOAD_MASS__KG_, Landing_Outcome from SPACEXTBL
WHERE PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000
AND Landing_Outcome = 'Success (drone ship)';

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_,Landing_Outcome
F9 FT B1022,4696,Success (drone ship)
F9 FT B1026,4600,Success (drone ship)
F9 FT B1021.2,5300,Success (drone ship)
F9 FT B1031.2,5200,Success (drone ship)


##### List the total number of successful and failure mission outcomes

In [48]:
%%sql
SELECT Landing_Outcome, COUNT(Landing_Outcome)
FROM SPACEXTBL
GROUP BY Landing_Outcome;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,COUNT(Landing_Outcome)
Controlled (ocean),5
Failure,3
Failure (drone ship),5
Failure (parachute),2
No attempt,21
No attempt,1
Precluded (drone ship),1
Success,38
Success (drone ship),14
Success (ground pad),9


##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [49]:
%%sql
SELECT Booster_Version, PAYLOAD_MASS__KG_
FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_)
    FROM SPACEXTBL);

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**

In [53]:
%%sql
SELECT CASE substr(Date, 6, 2)
    WHEN '01' THEN 'January'
    WHEN '02' THEN 'February'
    WHEN '03' THEN 'March'
    WHEN '04' THEN 'April'
    WHEN '05' THEN 'May'
    WHEN '06' THEN 'June'
    WHEN '07' THEN 'July'
    WHEN '08' THEN 'August'
    WHEN '09' THEN 'September'
    WHEN '10' THEN 'October'
    WHEN '11' THEN 'November'
    WHEN '12' THEN 'December'
    END as month_name, Landing_Outcome, Booster_Version, Launch_Site
from SPACEXTBL
WHERE substr(Date,0,5)='2015'
AND Landing_Outcome = 'Failure (drone ship)'

 * sqlite:///my_data1.db
Done.


month_name,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [56]:
%%sql
SELECT Landing_Outcome, COUNT(Landing_Outcome) as Count
FROM SPACEXTBL
WHERE Date BETWEEN '2010-06-04' and '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Count DESC

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
